In [5]:
"""
Stage-24  ─  End-to-end, in-memory RISE snapshot
================================================

What this *single* script does
------------------------------
1.  **Re-runs** the key logic of Stages 01 → 09 *in RAM* (no intermediate
    artefacts are read or written) for the hard-coded ASX ticker list and the
    calendar event-years 2018 → 2024.
2.  Collects each year’s blended resilience probability
    (`Mean_RISE_prob`) into one wide table (rows=tickers, columns=RISE_<year>).
3.  Pretty-prints the table.  **Absolutely nothing is persisted to disk.**

Key assumption
--------------
The low-level helper functions/classes used by your pipeline **exist as
importable Python symbols**—exactly the same ones Stage-01, 03, 06 and 09
already call.  If their actual module paths differ in your repo, simply
tweak the four import lines near the top of the file.

Ticker universe (kept strictly in RAM)
--------------------------------------
AD8 ARB C79 CAR CBA COH CSL CTD DMP FCL FPH GQG GYG HUB IEL JDO JHX LOV
MP1 MQG NAN NWL NXL PWH REA RIO RMD SEK SQ2 WTC XRO
"""
from __future__ import annotations

import warnings
from datetime import datetime
import pandas as pd

# ────────────────────────────────────────────────────────────────────── #
# 0.  STATIC UNIVERSE – never written anywhere
# ────────────────────────────────────────────────────────────────────── #
TICKERS: list[str] = [
    "AD8","ARB","C79","CAR","CBA","COH","CSL","CTD","DMP","FCL","FPH","GQG","GYG",
    "HUB","IEL","JDO","JHX","LOV","MP1","MQG","NAN","NWL","NXL","PWH","REA","RIO",
    "RMD","SEK","SQ2","WTC","XRO"
]
YEARS = range(2018, 2025)    # 2018-2024 inclusive

# ────────────────────────────────────────────────────────────────────── #
# 1.  Import the *same* building blocks the earlier stages use
#     (Rename these four lines if your repo uses different paths.)
# ────────────────────────────────────────────────────────────────────── #
try:
    # Stage-01 equivalent: returns cleaned & typed company financials
    from stage01_loader import load_and_clean                         # noqa: E402
    # Stage-03 equivalent: generates winsorised ratio matrix
    from stage03_ratios import build_ratio_matrix                     # noqa: E402
    # Stage-06 equivalent: turns ratios → probability per domain/stage
    from stage06_models import domain_stage_weighted_probs            # noqa: E402
    # Stage-09 equivalent: combines all prob columns → Mean_RISE_prob
    from stage09_blender import blended_rise_probability              # noqa: E402
except ImportError as e:  # FRIENDLY message so users know what to fix
    raise ImportError(
        "\nStage-24 could not find one or more helper modules.\n"
        "Please adjust the four import lines at the top of this file so they\n"
        "point to the actual Stage-01 / Stage-03 / Stage-06 / Stage-09 helpers\n"
        "in *your* repository."
    ) from e

# ────────────────────────────────────────────────────────────────────── #
# 2.  Mini-pipeline for each single EVENT_YEAR
# ────────────────────────────────────────────────────────────────────── #
def rise_column_for_year(event_year: int) -> pd.DataFrame:
    """
    Run all logic (01→03→06→09) for <event_year> but only for TICKERS.
    Returns a DataFrame indexed by 'Ticker' with *one* column:
        RISE_<event_year>
    """
    # -- Stage-01 logic -----------------------------------------------------
    df_clean = load_and_clean(event_year=event_year)

    # filter to our 31 tickers (case-insensitive robustness)
    df_clean["Ticker"] = df_clean["Ticker"].str.upper()
    df_clean = df_clean[df_clean["Ticker"].isin(TICKERS)]

    if df_clean.empty:
        warnings.warn(f"[{event_year}] none of the requested tickers found.")
        return pd.DataFrame({"Ticker": TICKERS, f"RISE_{event_year}": pd.NA})\
                 .set_index("Ticker")

    # -- Stage-03 logic -----------------------------------------------------
    df_ratios = build_ratio_matrix(df_clean, event_year=event_year)

    # -- Stage-06 logic -----------------------------------------------------
    df_probs  = domain_stage_weighted_probs(df_ratios, event_year=event_year)

    # -- Stage-09 logic -----------------------------------------------------
    df_blend  = blended_rise_probability(df_probs)

    # isolate ticker + Mean_RISE_prob → rename to RISE_<year>
    col_name = f"RISE_{event_year}"
    return (df_blend[["Ticker", "Mean_RISE_prob"]]
            .rename(columns={"Mean_RISE_prob": col_name})
            .set_index("Ticker"))

# ────────────────────────────────────────────────────────────────────── #
# 3.  Build the wide table across years
# ────────────────────────────────────────────────────────────────────── #
wide = pd.DataFrame(index=pd.Index(TICKERS, name="Ticker"))

for yr in YEARS:
    wide = wide.join(rise_column_for_year(yr), how="left")

# ensure chronological column order
wide = wide[[f"RISE_{y}" for y in YEARS]]

# ────────────────────────────────────────────────────────────────────── #
# 4.  Display – nothing is written to disk
# ────────────────────────────────────────────────────────────────────── #
print("\n================  RISE probabilities 2018 → 2024  ================\n")
print(wide.to_string(float_format=lambda x: f"{x:.3f}" if pd.notna(x) else "nan"))
print("\n==================================================================\n")
print(f"Generated {datetime.utcnow():%Y-%m-%d %H:%M:%S} UTC")

ImportError: 
Stage-24 could not find one or more helper modules.
Please adjust the four import lines at the top of this file so they
point to the actual Stage-01 / Stage-03 / Stage-06 / Stage-09 helpers
in *your* repository.